In [204]:
from keras import Input
from keras import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.datasets import make_classification
import pandas as pd
import csv


In [156]:
Y_POS =   8

# fix random seed for reproducibility

np.random.seed(7)


df = pd.read_csv('Diabetes (Edited).csv')
print(df)
print(df.describe())



      A    B   C   D    E     F      G   H  I
0     6  148  72  35    0  33.6  0.627  50  1
1     1   85  66  29    0  26.6  0.351  31  0
2     8  183  64   0    0  23.3  0.672  32  1
3     1   89  66  23   94  28.1  0.167  21  0
4     0  137  40  35  168  43.1  2.288  33  1
..   ..  ...  ..  ..  ...   ...    ...  .. ..
763  10  101  76  48  180  32.9  0.171  63  0
764   2  122  70  27    0  36.8  0.340  27  0
765   5  121  72  23  112  26.2  0.245  30  0
766   1  126  60   0    0  30.1  0.349  47  1
767   1   93  70  31    0  30.4  0.315  23  0

[768 rows x 9 columns]
                A           B           C           D           E           F  \
count  768.000000  768.000000  768.000000  768.000000  768.000000  768.000000   
mean     3.845052  120.894531   69.105469   20.536458   79.799479   31.992578   
std      3.369578   31.972618   19.355807   15.952218  115.244002    7.884160   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000  

In [157]:
X = df.iloc[:,0:Y_POS]
Y = df.iloc[:,Y_POS]

# create thee models
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=2020)

#scaling to around -2 to 2 (Z)
scaler = preprocessing.StandardScaler().fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [158]:
def train_and_predict_using_model(model_name = "", model =None):
    model.fit(scaled_X_train, Y_train)
    Y_pred_train = model.predict(scaled_X_train)
    cm_train = confusion_matrix(Y_train, Y_pred_train)
    print(model_name)
    print("====================================")
    print("Training Confusion Matrix: ")
    print(cm_train)
    acc_train = (cm_train[0,0] + cm_train[1,1]) / np.sum(np.sum(cm_train))
    
    print("Training Accuracy: %.2f%%" % (acc_train*100))
    print("====================================")
    
    Y_pred = model.predict(scaled_X_test)
    cm_test = confusion_matrix(Y_test, Y_pred)
    print("Testing Confusion Matrix: ")
    print(cm_test)
    acc_test = (cm_test[0,0] + cm_test[1,1]) / np.sum(np.sum(cm_test))
    
    print("Testing Accuracy: %.2f%%" % (acc_test*100))
    print("====================================")

In [159]:
linear_classifier = linear_model.LogisticRegression(random_state=123)
linear_classifier.fit(scaled_X_train, Y_train)

LogisticRegression(random_state=123)

In [160]:
Y_pred_train1 = linear_classifier.predict(scaled_X_train)
cm1_train = confusion_matrix(Y_train, Y_pred_train1)
print("Logistic Regression")
print("====================================")
print("Training Confusion Matrix: ")
print(cm1_train)
acc1_train = (cm1_train[0,0] + cm1_train[1,1]) / np.sum(np.sum(cm1_train))

print("Training Accuracy: %.2f%%" % (acc1_train*100))
print("====================================")

Y_pred1 = linear_classifier.predict(scaled_X_test)
cm1_test = confusion_matrix(Y_test, Y_pred1)
print("Testing Confusion Matrix: ")
print(cm1_test)
acc1_test = (cm1_test[0,0] + cm1_test[1,1]) / np.sum(np.sum(cm1_test))

print("Testing Accuracy: %.2f%%" % (acc1_test*100))
print("====================================")



Logistic Regression
Training Confusion Matrix: 
[[274  31]
 [ 62  93]]
Training Accuracy: 79.78%
Testing Confusion Matrix: 
[[172  23]
 [ 53  60]]
Testing Accuracy: 75.32%


In [161]:
train_and_predict_using_model('Logistic Regression', linear_classifier)

Logistic Regression
Training Confusion Matrix: 
[[274  31]
 [ 62  93]]
Training Accuracy: 79.78%
Testing Confusion Matrix: 
[[172  23]
 [ 53  60]]
Testing Accuracy: 75.32%


In [162]:
# Decision Tree and Random Forests
decision_tree_clf = tree.DecisionTreeClassifier()
train_and_predict_using_model('Decision Tree', decision_tree_clf)

Decision Tree
Training Confusion Matrix: 
[[305   0]
 [  0 155]]
Training Accuracy: 100.00%
Testing Confusion Matrix: 
[[137  58]
 [ 55  58]]
Testing Accuracy: 63.31%


In [163]:
# Neural Network
nn_model = Sequential()

# Add an Input layer with shape (8,) nn_model.add(Dense(8, activation='relu')) nn_model.add(Input(shape=(8,))) nn_model.add(Dense(8)) 

nn_model.add(Input(shape=(8,))) 

# Add a Dense layer with 5 outputs and 'relu' activation function
nn_model.add(Dense(5, activation='relu'))

# Add a final Dense layer with 1 output and 'sigmoid' activation function
nn_model.add(Dense(1, activation='sigmoid'))

In [164]:
# Compile model
nn_model.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])

In [165]:
# Fit the Model

nn_model.fit(scaled_X_train, Y_train, epochs=1, batch_size=20, verbose=0)


In [166]:
#Evaluate the model
scores = nn_model.evaluate(scaled_X_train, Y_train)

print("Neural Network Trainset: \n%s: %.2f%%" % (nn_model.metrics_names[1], scores[1]*100))
predictions = nn_model.predict(scaled_X_test)
Y_pred = (predictions > 0.5)
Y_pred = Y_pred*1
cm = confusion_matrix(Y_test, Y_pred)
print("====================================")
print("====================================")
print("Neural Network Testset Confusion Matrix: ")
print(cm)

acc = (cm[0,0] + cm[1,1]) / np.sum(np.sum(cm))
print("Neural Network Testset Accuracy: %.2f%%" % (acc*100))

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - accuracy: 0.6224 - loss: 0.7244
Neural Network Trainset: 
compile_metrics: 61.52%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Neural Network Testset Confusion Matrix: 
[[120  75]
 [ 60  53]]
Neural Network Testset Accuracy: 56.17%


In [167]:
# Neural Network reWork
model = Sequential()
model.add(Input(shape=(Y_POS,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

In [168]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [169]:
# Fit the Model
model.fit(scaled_X_train, Y_train, epochs=200, batch_size=20, verbose=0)


In [170]:
# Evaluate the Model 

scores = model.evaluate(scaled_X_train, Y_train)
print("Neural Network Trainset: \n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

predictions = model.predict(scaled_X_test)
Y_pred = (predictions > 0.5)
Y_pred = Y_pred*1
cm = confusion_matrix(Y_test, Y_pred)
print("====================================")
print("====================================")
print("Neural Network Testset Confusion Matrix: ")
print(cm)

acc = (cm[0,0] + cm[1,1]) / np.sum(np.sum(cm))
print("Neural Network Testset Accuracy: %.2f%%" % (acc*100))

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - accuracy: 0.9974 - loss: 0.0146
Neural Network Trainset: 
compile_metrics: 99.78%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Neural Network Testset Confusion Matrix: 
[[151  44]
 [ 51  62]]
Neural Network Testset Accuracy: 69.16%


In [171]:
#model 3 Random Forest

model3 = RandomForestRegressor()
model3.fit(X_train, Y_train)
print("Random Forest Regressor")
print("============================================================")
Y_pred = model3.predict(X_test)
RMSE__test = mean_squared_error(Y_test, Y_pred)

print("Random Forest Test set: RMSE {}".format(RMSE__test))

Random Forest Regressor
Random Forest Test set: RMSE 0.1698386363636364


In [172]:
importances = model3.feature_importances_

std = np.std([tree.feature_importances_ for tree in model3.estimators_], axis=0)

indices = np.argsort(importances)[::-1]



Chapter 11.8 Top 3 Features


In [192]:
indices_top3 = indices[:3]

np.random.seed(7)

Y_POS = 8

TOP_N_FEATURES = 3

X = df.iloc[:,0:TOP_N_FEATURES]
Y = df.iloc[:,Y_POS]

# create thee models
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2020)

#scaling to around -2 to 2 (Z)
scaler = preprocessing.StandardScaler().fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [193]:
# Model 1: Linear Regression

linear_clsaasifier = linear_model.LogisticRegression(random_state=123)
linear_classifier.fit(scaled_X_train, Y_train)

Y_pred_train1 = linear_classifier.predict(scaled_X_train)
cm1_train = confusion_matrix(Y_train, Y_pred_train1)
print("Regression")
print("====================================")
print("Training Confusion Matrix: ")
print(cm1_train)
acc1_train = (cm1_train[0,0] + cm1_train[1,1]) / np.sum(np.sum(cm1_train))

print("Training Accuracy: %.2f%%" % (acc1_train*100))
print("====================================")

Y_pred1 = linear_classifier.predict(scaled_X_test)
cm1_test = confusion_matrix(Y_test, Y_pred1)
print("Testing Confusion Matrix: ")
print(cm1_test)
acc1_test = (cm1_test[0,0] + cm1_test[1,1]) / np.sum(np.sum(cm1_test))

print("Testing Accuracy: %.2f%%" % (acc1_test*100))
print("====================================")


Regression
Training Confusion Matrix: 
[[360  47]
 [103 104]]
Training Accuracy: 75.57%
Testing Confusion Matrix: 
[[83 10]
 [31 30]]
Testing Accuracy: 73.38%


In [194]:
# Model 2: Decision Tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(scaled_X_train, Y_train)
Y_pred_train2 = clf.predict(scaled_X_train)
cm2_train = confusion_matrix(Y_train, Y_pred_train2)
print("Decision Tree")
print("====================================")
print("Training Confusion Matrix: ")
print(cm2_train)
acc2_train = (cm2_train[0,0] + cm2_train[1,1]) / np.sum(np.sum(cm2_train))

print("Training Accuracy: %.2f%%" % (acc2_train*100))
print("====================================")

Y_pred2 = clf.predict(scaled_X_test)    
cm2_test = confusion_matrix(Y_test, Y_pred2)
print("Testing Confusion Matrix: ")
print(cm2_test)
acc2_test = (cm2_test[0,0] + cm2_test[1,1]) / np.sum(np.sum(cm2_test))

print("Testing Accuracy: %.2f%%" % (acc2_test*100))
print("====================================")



Decision Tree
Training Confusion Matrix: 
[[407   0]
 [  3 204]]
Training Accuracy: 99.51%
Testing Confusion Matrix: 
[[60 33]
 [27 34]]
Testing Accuracy: 61.04%


In [195]:
# Model 3: Random Forest

model3 = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
model3.fit(scaled_X_train, Y_train)
Y_pred_train3 = model3.predict(scaled_X_train)
cm3_train = confusion_matrix(Y_train, Y_pred_train3)
print("Random Forest")
print("====================================")
print("Training Confusion Matrix: ")
print(cm3_train)
acc3_train = (cm3_train[0,0] + cm3_train[1,1]) / np.sum(np.sum(cm3_train))

print("Training Accuracy: %.2f%%" % (acc3_train*100))
print("====================================")

Y_pred3 = model3.predict(scaled_X_test)
cm3_test = confusion_matrix(Y_test, Y_pred3)
print("Testing Confusion Matrix: ")
print(cm3_test)
acc3_test = (cm3_test[0,0] + cm3_test[1,1]) / np.sum(np.sum(cm3_test))

print("Testing Accuracy: %.2f%%" % (acc3_test*100))
print("====================================")



Random Forest
Training Confusion Matrix: 
[[388  19]
 [131  76]]
Training Accuracy: 75.57%
Testing Confusion Matrix: 
[[88  5]
 [36 25]]
Testing Accuracy: 73.38%


In [196]:
# Model 4: Gradient Boosting

model4 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model4.fit(scaled_X_train, Y_train)
Y_pred_train4 = model4.predict(scaled_X_train)
cm4_train = confusion_matrix(Y_train, Y_pred_train4)
print("Gradient Boosting")
print("====================================")
print("Training Confusion Matrix: ")
print(cm4_train)
acc4_train = (cm4_train[0,0] + cm4_train[1,1]) / np.sum(np.sum(cm4_train))
print("Training Accuracy: %.2f%%" % (acc4_train*100))
print("====================================")

preds = model4.predict(scaled_X_test)
Y_pred4 = (preds > 0.5)
Y_pred4 = Y_pred4*1
cm4_test = confusion_matrix(Y_test, Y_pred4)
print("Testing Confusion Matrix: ")
print(cm4_test)
acc4_test = (cm4_test[0,0] + cm4_test[1,1]) / np.sum(np.sum(cm4_test))
print("Testing Accuracy: %.2f%%" % (acc4_test*100))
print("====================================")


Gradient Boosting
Training Confusion Matrix: 
[[359  48]
 [ 86 121]]
Training Accuracy: 78.18%
Testing Confusion Matrix: 
[[77 16]
 [33 28]]
Testing Accuracy: 68.18%


In [197]:
# Model 5: Neural Network
model5 = Sequential()
model5.add(Input(shape=(TOP_N_FEATURES,)))
model5.add(Dense(10, activation='relu'))
model5.add(Dense(10, activation='relu'))
model5.add(Dropout(0.2))
model5.add(Dense(1, activation='sigmoid'))

# Compile model
model5.compile(loss='binary_crossentropy', optimizer='Adamax', metrics=['accuracy'])

model5.fit(X_train, Y_train, epochs=100, batch_size=5, verbose=0)

# Evaluate the model
scores = model5.evaluate(X_train, Y_train)

print("Neural Network Trainset: \n%s: %.2f%%" % (model5.metrics_names[1], scores[1]*100))

predictions = model5.predict(X_test)

Y_pred = (predictions > 0.5)
Y_pred = Y_pred*1
cm = confusion_matrix(Y_test, Y_pred)
print("Neural Network Testset Confusion Matrix: ")
print(cm)

acc = (cm[0,0] + cm[1,1]) / np.sum(np.sum(cm))
print("Neural Network Testset Accuracy: %.2f%%" % (acc*100))



20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - accuracy: 0.7164 - loss: 0.5696
Neural Network Trainset: 
compile_metrics: 73.13%
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Neural Network Testset Confusion Matrix: 
[[80 13]
 [39 22]]
Neural Network Testset Accuracy: 66.23%


In [198]:
from sklearn import svm

clf = svm.SVC()
train_and_predict_using_model('SVM (Classiefier)', clf)

SVM (Classiefier)
Training Confusion Matrix: 
[[374  33]
 [101 106]]
Training Accuracy: 78.18%
Testing Confusion Matrix: 
[[85  8]
 [36 25]]
Testing Accuracy: 71.43%


Chapter 11.9 Hyper Parameters and SVM

In [199]:


rbf_clf = svm.SVC(kernel='rbf')
train_and_predict_using_model('SVM (Classiefier) - RBF', rbf_clf)

SVM (Classiefier) - RBF
Training Confusion Matrix: 
[[374  33]
 [101 106]]
Training Accuracy: 78.18%
Testing Confusion Matrix: 
[[85  8]
 [36 25]]
Testing Accuracy: 71.43%


In [200]:
rbf_svc = svm.SVC(kernel='poly')
train_and_predict_using_model('SVM (Classiefier) - Poly', rbf_svc)

SVM (Classiefier) - Poly
Training Confusion Matrix: 
[[389  18]
 [139  68]]
Training Accuracy: 74.43%
Testing Confusion Matrix: 
[[88  5]
 [43 18]]
Testing Accuracy: 68.83%


In [201]:
rbf_svc = svm.SVC(kernel='sigmoid')
train_and_predict_using_model('SVM (Classiefier) - Sigmoid', rbf_svc)

SVM (Classiefier) - Sigmoid
Training Confusion Matrix: 
[[310  97]
 [106 101]]
Training Accuracy: 66.94%
Testing Confusion Matrix: 
[[70 23]
 [32 29]]
Testing Accuracy: 64.29%


In [202]:
wclf = svm.SVC(kernel='linear', class_weight={1:2})
train_and_predict_using_model('SVM (Classiefier) - Linear', wclf)

SVM (Classiefier) - Linear
Training Confusion Matrix: 
[[320  87]
 [ 70 137]]
Training Accuracy: 74.43%
Testing Confusion Matrix: 
[[70 23]
 [19 42]]
Testing Accuracy: 72.73%


Chapter 11.10 Naive Bayes


In [203]:
from sklearn.naive_bayes import GaussianNB

# Maximum Likelihood Estimation

gnb = GaussianNB()
train_and_predict_using_model('Naive Bayes', gnb)

Naive Bayes
Training Confusion Matrix: 
[[355  52]
 [ 94 113]]
Training Accuracy: 76.22%
Testing Confusion Matrix: 
[[80 13]
 [28 33]]
Testing Accuracy: 73.38%


In [205]:
X, Y = make_classification(n_samples=1000, weights=[0.1, 0.9])
# Your GNB Estimator
gnb = GaussianNB()
gnb.fit(X, Y)

print("model prior {} close to your defined prior {}".format(gnb.class_prior_, [0.1, 0.9]))

model prior [0.104 0.896] close to your defined prior [0.1, 0.9]


Chapter 11.11 Sample Code
